In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

In [5]:
train_dir = 'chest/train'
val_dir = 'chest/val'
test_dir = 'chest/test'

# Buat image data generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [6]:
model = Sequential([
    # Layer Konvolusi pertama
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),

    # Layer Konvolusi kedua
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    # Layer Konvolusi ketiga untuk menangkap fitur yang lebih kompleks
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    # Layer Konvolusi keempat
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    # Meratakan output dari layer konvolusi menjadi 1D
    Flatten(),

    # Menambahkan Dropout untuk mencegah overfitting
    Dropout(0.5),

    # Layer Dense (fully-connected)
    Dense(512, activation='relu'),

    # Layer output dengan aktivasi sigmoid untuk klasifikasi biner (Normal/Pneumonia)
    Dense(1, activation='sigmoid')
])


model.summary()

d:\project ngoding\fullstack pneumonia app detect\model\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     3,211,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,453,121 (13.17 MB)

 Trainable params: 3,453,121 (13.17 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Mengompilasi model dengan optimizer 'adam', loss function 'binary_crossentropy'
# dan metrik 'accuracy'
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
# Melatih model dengan data generator
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size, # Jumlah batch per epoch
    epochs=25, # Jumlah epoch (bisa disesuaikan)
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size # Jumlah batch validasi
)

d:\project ngoding\fullstack pneumonia app detect\model\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ImportError: This requires the scipy module. You can install it via `pip install scipy`

In [ ]:
# Mengambil nilai akurasi dan loss dari history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

# Membuat plot untuk Akurasi
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Akurasi Training')
plt.plot(epochs_range, val_acc, label='Akurasi Validasi')
plt.legend(loc='lower right')
plt.title('Akurasi Training dan Validasi')

# Membuat plot untuk Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Loss Training')
plt.plot(epochs_range, val_loss, label='Loss Validasi')
plt.legend(loc='upper right')
plt.title('Loss Training dan Validasi')
plt.show()

In [ ]:
def predict_image(img_path):
    """Fungsi untuk memprediksi gambar tunggal."""
    # 1. Memuat dan mengubah ukuran gambar
    img = image.load_img(img_path, target_size=(150, 150))

    # 2. Mengubah gambar menjadi array numpy dan menambahkan dimensi batch
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    # 3. Normalisasi gambar (penting!)
    x = x / 255.0

    # 4. Melakukan prediksi
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    
    # 5. Menampilkan hasil prediksi
    print(f"File: {img_path}")
    print(f"Nilai Prediksi: {classes[0][0]}")
    
    if classes[0] > 0.5:
        print("Hasil: Terdeteksi PNEUMONIA")
    else:
        print("Hasil: NORMAL")

# --- Contoh Penggunaan ---
# Ganti 'path/to/your/image.jpeg' dengan path file gambar yang ingin Anda uji.
# Misalnya, Anda bisa mengambil satu gambar dari folder 'chest/test/NORMAL' atau 'chest/test/PNEUMONIA'
try:
    predict_image('chest\test\NORMAL\IM-0001-0001.jpeg')
    print("-" * 30)
    predict_image('chest\test\NORMAL\IM-0001-0001.jpeg')
except FileNotFoundError:
    print("\nPastikan path file gambar sudah benar!")
    print("Misalnya: 'chest/test/NORMAL/nama_file.jpeg'")